TODO: confirm FALSE->FAKE mapping is correct (swap to test)

TODO: try to use cuml environment

TODO: Learn and implement Random Forest

Data From: https://github.com/jpposadas/FakeNewsCorpusSpanish/

(spanish_fake_news_1.csv from train.csv; spanish_fake_news_2.csv from test.csv: manually created union to repartition)

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from lib import *
from NLP_algorithms import *

In [3]:
# Merge two Spanish fake news datasets into one dataframe

spanish_fake_news_1 = pd.read_csv('data/spanish_fake_news_1.csv')
spanish_fake_news_2 = pd.read_csv('data/spanish_fake_news_2.csv')

# Change all of the values in CATEGORY of fake_news_2 from 'FALSE' to 'Fake' and 'TRUE' to 'True'
spanish_fake_news_2['CATEGORY'] = spanish_fake_news_2['CATEGORY'].replace({False: 'Fake', True: 'True'})

#Rename the columns of fake_news_2 to match those of fake_news_1
spanish_fake_news_2.rename(columns={'CATEGORY': 'Category', 'TITLE': 'Title', 'TEXT': 'Text', "ID" : "Id", "HEADLINE": "Headline", "LINK": "Link", "SOURCE": "Source", "TOPICS" : "Topic"}, inplace=True)

# Concatenate the two dataframes into one
spanish_fake_news = pd.concat([spanish_fake_news_1, spanish_fake_news_2], ignore_index=True)
#display(spanish_fake_news.tail())

# Map FAKE->1 and TRUE->0
spanish_fake_news['Category'] = spanish_fake_news['Category'].replace({'Fake': 1, 'True': 0})

# View first and last 5 rows
display(spanish_fake_news.head())
display(spanish_fake_news.tail())

/var/folders/tp/3nd20r2x6ns2txnn5kmx15dr0000gn/T/ipykernel_93062/1480579410.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spanish_fake_news['Category'] = spanish_fake_news['Category'].replace({'Fake': 1, 'True': 0})


,Id,Category,Topic,Source,Headline,Text,Link
0,1,1,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,2,1,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,3,1,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,4,0,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,5,1,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


,Id,Category,Topic,Source,Headline,Text,Link
1243,568,0,Covid-19,El Financiero,Encuentran nueva variante de COVID en México: ...,El Instituto de Diagnóstico y Referencia Epide...,https://www.elfinanciero.com.mx/salud/encuentr...
1244,569,1,Sociedad,diariogol,El móvil de más de 60.000 euros de la princesa...,La hija del rey Felipe y de la reina Letizia y...,https://www.diariogol.com/gossip/el-movil-de-m...
1245,570,1,Política,AFPFactual,NaN,"Evidentemente, Barak Obama ha sido arrestado e...","Perma | Obama, Biden y la directora de la CIA,..."
1246,571,0,Covid-19,Redacción Médica,Covid: las vacunas puestas en España no alcanz...,El Ministerio de Sanidad ha actualizado los da...,https://www.redaccionmedica.com/secciones/sani...
1247,572,1,Covid-19,El Sol de la Laguna,En 2014 una revista ya alertaba sobre el coron...,Fue en el mes de febrero de 2014 cuando la rev...,https://www.Noticiadelsoldelalaguna.com.mx/dob...
